In [4]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')
# ---

%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 100
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np

from sklearn import base

pd.options.display.max_rows = 100

In [5]:
pwd

u'C:\\Users\\cnyi\\Box Sync\\Kaggle\\titanic'

In [6]:
#data.describe() # Age is missing

In [7]:
#data.head(10)

In [8]:
from sklearn import base

class Createdummy(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self

    
    def transform(self, X):
        import pandas as pd
        X=pd.DataFrame(X)
        new_data=pd.get_dummies(X[self.column_name])
        new_data=X.join(new_data)
        new_data.drop(self.column_name,1,inplace=True)
        return new_data

In [9]:
from sklearn import base

class CreateTitle(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self

    
    def transform(self,X):    
        # we extract the title from each name
        X['Title'] = X[self.column_name].map(lambda name:name.split(',')[1].split('.')[0].strip())

        # a map of more aggregated titles
        Title_Dictionary = {
                            "Capt":       "Officer",
                            "Col":        "Officer",
                            "Major":      "Officer",
                            "Jonkheer":   "Royalty",
                            "Don":        "Royalty",
                            "Sir" :       "Royalty",
                            "Dr":         "Officer",
                            "Rev":        "Officer",
                            "the Countess":"Royalty",
                            "Dona":       "Royalty",
                            "Mme":        "Mrs",
                            "Mlle":       "Miss",
                            "Ms":         "Mrs",
                            "Mr" :        "Mr",
                            "Mrs" :       "Mrs",
                            "Miss" :      "Miss",
                            "Master" :    "Master",
                            "Lady" :      "Royalty"

                            }

        # we map each title
        X['Title'] = X.Title.map(Title_Dictionary)
        return X

In [10]:
from sklearn import base

class dropcolumn(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self

    
    def transform(self,X):
        data_new=X.drop(self.column_name,1)
        return data_new
        

In [11]:
from sklearn import base

class PrepareData(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        try:
            X=X[X.columns.drop(self.column_name)]
        #return X
            return X.values
        except:
            return X.values
        


In [12]:
from sklearn import base

class fill_nan(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.column_name].fillna(X[self.column_name].median(), inplace=True)
        return X
   

In [13]:
data = pd.read_csv('data/train.csv')

In [14]:
data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [108]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

titanic_LinearSVC=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('LinearSVC',LinearSVC())
    ])

LinearSVC_parameters = {
    'LinearSVC__C':  np.arange(0.5, 3, 0.25)  
}

In [109]:
from sklearn.model_selection import GridSearchCV    
grid_search = GridSearchCV(titanic_LinearSVC, LinearSVC_parameters, verbose=1)

print("Performing grid search...")
print("titanic_LinearSVC:", [name for name, _ in titanic_LinearSVC.steps])
print("parameters:")
grid_search.fit(X=data, y=data['Survived'].values)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(LinearSVC_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
('titanic_LinearSVC:', ['fill_nan', 'title', 'drop', 'dummy', 'PrepareData', 'LinearSVC'])
parameters:
Fitting 3 folds for each of 10 candidates, totalling 30 fits
()
Best score: 0.813
Best parameters set:
	LinearSVC__C: 0.75


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   15.1s finished


In [110]:
titanic_LinearSVC_best=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('LinearSVC',LinearSVC(C=0.75))
    ])


In [111]:
titanic_LinearSVC_best.fit(data,data['Survived'].values)

Pipeline(steps=[('fill_nan', fill_nan(column_name='Age')), ('title', CreateTitle(column_name='Name')), ('drop', dropcolumn(column_name=['Name', 'Ticket', 'Cabin', 'PassengerId'])), ('dummy', Createdummy(column_name=['Title', 'Sex', 'Embarked'])), ('PrepareData', PrepareData(column_name='Survived')), ('Linear...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [57]:
titanic_LinearSVC_best.steps[5][1].feature_importances_

AttributeError: 'LinearSVC' object has no attribute 'feature_importances_'

In [112]:
from sklearn.ensemble import RandomForestClassifier
titanic_rf=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('rf',RandomForestClassifier())
    ])

rf_parameters = {
    'rf__max_depth':  range(2,15)
}

In [113]:
from sklearn.model_selection import GridSearchCV    
grid_search = GridSearchCV(titanic_rf, rf_parameters, verbose=1)

print("Performing grid search...")
print("titanic_rf:", [name for name, _ in titanic_rf.steps])
print("parameters:")
grid_search.fit(X=data, y=data['Survived'].values)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(rf_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
('titanic_rf:', ['fill_nan', 'title', 'drop', 'dummy', 'PrepareData', 'rf'])
parameters:
Fitting 3 folds for each of 13 candidates, totalling 39 fits
()
Best score: 0.825
Best parameters set:
	rf__max_depth: 9


[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:   22.1s finished


In [114]:
titanic_rf_best=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('rf',RandomForestClassifier(max_depth= 9))
    ])


In [67]:
titanic_rf_best.fit(data,data['Survived'].values)

Pipeline(steps=[('fill_nan', fill_nan(column_name='Age')), ('title', CreateTitle(column_name='Name')), ('drop', dropcolumn(column_name=['Name', 'Ticket', 'Cabin', 'PassengerId'])), ('dummy', Createdummy(column_name=['Title', 'Sex', 'Embarked'])), ('PrepareData', PrepareData(column_name='Survived')), ('rf', R...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

In [68]:
titanic_rf_best.steps[5][1].feature_importances_

array([ 0.06628081,  0.21882364,  0.06318428,  0.03232479,  0.24121567,
        0.01400266,  0.01165633,  0.07382052,  0.02003714,  0.00270909,
        0.00040551,  0.17149669,  0.05284916,  0.01441784,  0.00637961,
        0.01039627])

In [102]:
range(2,10)

[2, 3, 4, 5, 6, 7, 8, 9]

In [104]:
from sklearn.neighbors import KNeighborsClassifier
titanic_knn=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('knn',KNeighborsClassifier())
    ])

knn_parameters = {
    'knn__n_neighbors':  range(2,100)
}


from sklearn.model_selection import GridSearchCV    
grid_search = GridSearchCV(titanic_knn, knn_parameters, verbose=1)

print("Performing grid search...")
print("titanic_knn:", [name for name, _ in titanic_knn.steps])
print("parameters:")
grid_search.fit(X=data, y=data['Survived'].values)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(knn_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
    

Performing grid search...
('titanic_knn:', ['fill_nan', 'title', 'drop', 'dummy', 'PrepareData', 'knn'])
parameters:
Fitting 3 folds for each of 98 candidates, totalling 294 fits
()
Best score: 0.715
Best parameters set:
	knn__n_neighbors: 5


[Parallel(n_jobs=1)]: Done 294 out of 294 | elapsed:  2.3min finished


In [115]:
titanic_knn_best=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('knn',KNeighborsClassifier(n_neighbors=5))
    ])

In [127]:
from sklearn.svm import SVC
titanic_SVC=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('SVC',SVC())
    ])

SVC_parameters = {
    'SVC__C':  np.arange(0.5,10,0.5)
}


from sklearn.model_selection import GridSearchCV    
grid_search = GridSearchCV(titanic_SVC, SVC_parameters, verbose=1)

print("Performing grid search...")
print("titanic_SVC:", [name for name, _ in titanic_SVC.steps])
print("parameters:")
grid_search.fit(X=data, y=data['Survived'].values)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(SVC_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
    

Performing grid search...
('titanic_SVC:', ['fill_nan', 'title', 'drop', 'dummy', 'PrepareData', 'SVC'])
parameters:
Fitting 3 folds for each of 19 candidates, totalling 57 fits
()
Best score: 0.751
Best parameters set:
	SVC__C: 4.5


[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed:   24.4s finished


In [128]:
titanic_SVC_best=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('SVC',SVC(C=4.5))
    ])


In [131]:
from sklearn.ensemble import GradientBoostingClassifier
titanic_GBT=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('GBT',GradientBoostingClassifier())
    ])

GBT_parameters = {
    'GBT__max_depth':  np.arange(2,20)
}


from sklearn.model_selection import GridSearchCV    
grid_search = GridSearchCV(titanic_GBT, GBT_parameters, verbose=1)

print("Performing grid search...")
print("titanic_GBT:", [name for name, _ in titanic_GBT.steps])
print("parameters:")
grid_search.fit(X=data, y=data['Survived'].values)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(GBT_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
    

Performing grid search...
('titanic_GBT:', ['fill_nan', 'title', 'drop', 'dummy', 'PrepareData', 'GBT'])
parameters:
Fitting 3 folds for each of 18 candidates, totalling 54 fits
()
Best score: 0.825
Best parameters set:
	GBT__max_depth: 4


[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:   42.9s finished


In [69]:
class EstimatorTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, estimator):
#         What needs to be done here?
#         self.estimator = CityEstimator()
        self.estimator = estimator
    
    def fit(self, X, y):
        
        # Fit the stored estimator.
        # Question: what should be returned?
        self.estimator.fit(X, y)
        return(self)
    
    def transform(self, X):
        # Use predict on the stored estimator as a "transformation".
        # Be sure to return a 2-D array.
        new_list = []
        new_array = self.estimator.predict(X)
        for i in new_array:
            ii = [i]
            new_list.append(ii)
        return(new_list)

In [150]:
from sklearn.pipeline import FeatureUnion

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier

pipeline_full = Pipeline([
    # Extract the subject & body
    ('fill_nan',fill_nan(['Age','Fare']))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))

    # Use FeatureUnion to combine the features from subject and body
    ,('union', FeatureUnion(
        transformer_list=[
               ('Ridge_best', EstimatorTransformer(LinearSVC(C=0.75)))

              ,('rf_best',EstimatorTransformer(RandomForestClassifier(max_depth= 9)))
            ,( 'knn_best', EstimatorTransformer(KNeighborsClassifier(n_neighbors=5)) )
            ,('SVC_best',EstimatorTransformer(SVC(C=4.5)))
            
            ,('GBT_best',EstimatorTransformer(GradientBoostingClassifier(max_depth=4)))
        ]
    #,transformer_weights={
    #        'Ridge_best': 1,
    #        'rf_best': 1,
    #}
    ))
   
    # Use a GBT Regressor on the combined features
    ,('MLPClassifier', MLPClassifier()),
    ])



MLPClassifier_parameters = {
    'MLPClassifier__alpha': np.arange(0.00001, 0.001, 0.0001)
    ,'MLPClassifier__hidden_layer_sizes': np.arange(1, 200)
}

In [ ]:
from sklearn.model_selection import GridSearchCV    
grid_search = GridSearchCV(pipeline_full, MLPClassifier_parameters, verbose=1)

print("Performing grid search...")
print("pipeline_full:", [name for name, _ in pipeline_full.steps])
print("parameters:")
grid_search.fit(X=data, y=data['Survived'].values)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(MLPClassifier_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
('pipeline_full:', ['fill_nan', 'title', 'drop', 'dummy', 'PrepareData', 'union', 'MLPClassifier'])
parameters:
Fitting 3 folds for each of 1990 candidates, totalling 5970 fits


In [120]:
pipeline_full_best = Pipeline([
    # Extract the subject & body
    ('fill_nan_Age',fill_nan('Age'))
    ,('fill_nan_Fare',fill_nan('Fare'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))

    # Use FeatureUnion to combine the features from subject and body
    ,('union', FeatureUnion(
        transformer_list=[
               ('Ridge_best', EstimatorTransformer(LinearSVC(C=0.75)))

              ,('rf_best',EstimatorTransformer(RandomForestClassifier(max_depth= 9)))
            
            ,( 'knn_best', EstimatorTransformer(KNeighborsClassifier(n_neighbors=5)) )
            
            ,('SVC_best',EstimatorTransformer(SVC(C=4.5)))
        ]
    #,transformer_weights={
    #        'Ridge_best': 1,
    #        'rf_best': 1,
    #}
    ))
   
    # Use a GBT Regressor on the combined features
    ,('GBT', GradientBoostingClassifier(max_depth= 2)),
    ])

In [121]:
pipeline_full_best.fit(X=data, y=data['Survived'].values)

Pipeline(steps=[('fill_nan_Age', fill_nan(column_name='Age')), ('fill_nan_Fare', fill_nan(column_name='Fare')), ('title', CreateTitle(column_name='Name')), ('drop', dropcolumn(column_name=['Name', 'Ticket', 'Cabin', 'PassengerId'])), ('dummy', Createdummy(column_name=['Title', 'Sex', 'Embarked'])), ('Prepare...=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False))])

In [122]:
test = pd.read_csv('data/test.csv')

test['Fare'].fillna(data['Fare'].median(), inplace=True)
test['Age'].fillna(data['Age'].median(), inplace=True)

In [84]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,29.805024,0.447368,0.392344,35.576535
std,120.810458,0.841838,12.667969,0.896760,0.981429,55.850103
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.471875
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [123]:
test_predit=pipeline_full_best.predict(test)

In [124]:
pd.DataFrame(test_predit).rename(columns = {0:'Survived'})

,Survived
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [125]:
test[['PassengerId']].join(pd.DataFrame(test_predit).rename(columns = {0:'Survived'})) 
test_predit_df=test[['PassengerId']].join(pd.DataFrame(test_predit).rename(columns = {0:'Survived'})) 

In [126]:
test_predit_df.to_csv('data/test_predit.csv',index=False)